In [3]:
import json
import pandas as pd

#from google.colab import drive
#drive.mount("/content/drive")

def read_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    data = [json.loads(line) for line in lines]
    return pd.DataFrame(data)

#upload data to the folder called "NLP data"
#evidence = read_json("/content/drive/My Drive/NLP data/evidence.json")
#dev_claims = read_json("/content/drive/My Drive/NLP data/dev-claims.json")
#dev_claims_baseline = read_json("/content/drive/My Drive/NLP data/dev-claims-baseline.json")
#test_claims_unlabelled = read_json("/content/drive/My Drive/NLP data/test-claims-unlabelled.json")
#train_claims = read_json("/content/drive/My Drive/NLP data/train-claims.json")

evidence_df = read_json("data/evidence.json")
dev_claims_df = read_json("data/dev-claims.json")
dev_claims_baseline_df = read_json("data/dev-claims-baseline.json")
test_claims_unlabelled_df = read_json("data/test-claims-unlabelled.json")
train_claims_df = read_json("data/train-claims.json")

In [24]:
print(evidence_df.shape)
print(dev_claims_df.shape)
print(train_claims_df.shape)
print(test_claims_unlabelled_df.shape)

(1, 1208827)
(1, 154)
(1, 1228)
(1, 153)


In [ ]:
print(dev_claims_df['claim-375'][0])
print(evidence_df['evidence-996421'][0])
print(evidence_df['evidence-1080858'][0])
print(evidence_df['evidence-208053'][0])
print(evidence_df['evidence-699212'][0])
print(evidence_df['evidence-832334'][0])

In [ ]:
dev_claims_base = dev_claims_baseline_df.T

dev_claims_base['evidence_count'] = dev_claims_base[0].apply(lambda x: len(x.get('evidences', [])))

max_evidence_count = dev_claims_base['evidence_count'].max()
min_evidence_count = dev_claims_base['evidence_count'].min()

print(dev_claims_base, max_evidence_count, min_evidence_count)

In [ ]:
dev_claims = dev_claims_df.T

dev_claims['evidence_count'] = dev_claims[0].apply(lambda x: len(x.get('evidences', [])))

max_evidence_count = dev_claims['evidence_count'].max()
min_evidence_count = dev_claims['evidence_count'].min()

print(dev_claims, max_evidence_count, min_evidence_count)

evidence_count_distribution = dev_claims['evidence_count'].value_counts()
evidence_count_distribution_sorted = evidence_count_distribution.sort_index()
print(evidence_count_distribution_sorted)

In [ ]:
train_claims = train_claims_df.T

train_claims['evidence_count'] = train_claims[0].apply(lambda x: len(x.get('evidences', [])))

max_evidence_count = train_claims['evidence_count'].max()
min_evidence_count = train_claims['evidence_count'].min()

print(train_claims, max_evidence_count, min_evidence_count)

### Data Structure Preprocessing (1-1)

In [3]:
train_claims_data = []
for claim_id, claim_info in train_claims_df.items():
    for evidence in claim_info[0]['evidences']:
        update_info = {
            "claim_ID": claim_id,
            "claim_text": claim_info[0]['claim_text'],
            "claim_label": claim_info[0]['claim_label'],
            "evidences": evidence
        }
        train_claims_data.append(update_info)

In [4]:
dev_claims_data = []
for claim_id, claim_info in dev_claims_df.items():
    for evidence in claim_info[0]['evidences']:
        update_info = {
            "claim_ID": claim_id,
            "claim_text": claim_info[0]['claim_text'],
            "claim_label": claim_info[0]['claim_label'],
            "evidences": evidence
        }
        dev_claims_data.append(update_info)

In [5]:
evidence_data = []

for col in evidence_df.columns:
    evidence_text = evidence_df[col][0]
    evidence_id = col
    evidence_data.append({
        "evidence_ID": evidence_id,
        "evidence_text": evidence_text
    })

In [ ]:
#print("train_claims_data: \n",train_claims_data[:3])
#print("dev_claims_data: \n",dev_claims_data[:3])
#print("evidence_data: \n",evidence_data[:3])

print("train_claims_data: \n",pd.DataFrame(train_claims_data).head(3))
print("dev_claims_data: \n",pd.DataFrame(dev_claims_data).head(3))
print("evidence_data: \n",pd.DataFrame(evidence_data).head(3))

In [ ]:
baseline_dev_claims_data = []
for claim_id, claim_info in dev_claims_baseline_df.items():
        update_info = {
            "claim_ID": claim_id,
            "claim_text": claim_info[0]['claim_text'],
            "claim_label": claim_info[0]['claim_label'],
            "evidences": claim_info[0]['evidences']
        }
        baseline_dev_claims_data.append(update_info)
baseline_dev_label_counts = pd.DataFrame(baseline_dev_claims_data)['claim_label'].value_counts()
print(baseline_dev_label_counts)

In [ ]:
dev_claims_data = []
for claim_id, claim_info in dev_claims_df.items():
        update_info = {
            "claim_ID": claim_id,
            "claim_text": claim_info[0]['claim_text'],
            "claim_label": claim_info[0]['claim_label'],
            "evidences": claim_info[0]['evidences']
        }
        dev_claims_data.append(update_info)
dev_label_counts = pd.DataFrame(dev_claims_data)['claim_label'].value_counts()
print(dev_label_counts)

In [ ]:
#label_counts = pd.DataFrame(train_claims_data)['claim_label'].value_counts()
dev_label_counts = pd.DataFrame(dev_claims_data)['claim_label'].value_counts()
print(dev_label_counts)

In [ ]:
train_label_counts = pd.DataFrame(train_claims_data)['claim_label'].value_counts()
print(train_label_counts)

### Data processing

In [ ]:
'''
import numpy as np
import tensorflow as tf

maxlen = 20

tokenizer = tf.keras.preprocessing.text.Tokenizer()

train_texts = [item['claim_text'] for item in train_claims_data]
tokenizer.fit_on_texts(train_texts)
train_claim_sequences = tokenizer.texts_to_sequences(train_texts)
#paddle
train_claim_data_pad = tf.keras.preprocessing.sequence.pad_sequences(train_claim_sequences, maxlen=maxlen)
train_claim_masks = np.where(train_claim_data != 0, 1, 0)
'''

In [ ]:
'''
evidence_texts = [item['evidence_text'] for item in evidence_data]
tokenizer.fit_on_texts(evidence_texts)
evidence_sequences = tokenizer.texts_to_sequences(evidence_texts)
#paddle
evidence_data_pad = tf.keras.preprocessing.sequence.pad_sequences(evidence_sequences, maxlen=maxlen)
#evidence_masks = np.where(evidence_data != 0, 1, 0)
'''

In [ ]:
#print(evidence_texts_pad[:5])
#print(evidence_data_pad[:5])

### TF-IDF & Cosine similarity

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text) 
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(filtered_words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yusi2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yusi2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yusi2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
#tfidf vectorize_without ngram
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

claims_texts = [preprocess_text(claim['claim_text']) for claim in train_claims_data]
evidence_texts = [preprocess_text(evidence['evidence_text']) for evidence in evidence_data]
all_texts = claims_texts + evidence_texts

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_texts)

In [ ]:
#tfidf vectorize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

claims_texts = [preprocess_text(claim['claim_text']) for claim in train_claims_data]
evidence_texts = [preprocess_text(evidence['evidence_text']) for evidence in evidence_data]
all_texts = claims_texts + evidence_texts

vectorizer_ngram = TfidfVectorizer(ngram_range=(1,2))
tfidf_matrix_ngram = vectorizer_ngram.fit_transform(all_texts)

In [ ]:
import pickle

# 儲存單詞向量和矩陣
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)
with open('tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# 儲存帶n-gram範圍的向量器和矩陣
with open('vectorizer_ngram.pkl', 'wb') as f:
    pickle.dump(vectorizer_ngram, f)
with open('tfidf_matrix_ngram.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix_ngram, f)

In [ ]:
# 加載向量器和矩陣
with open('vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)
with open('tfidf_matrix.pkl', 'rb') as f:
    loaded_tfidf_matrix = pickle.load(f)

# 加載帶n-gram範圍的向量器和矩陣
with open('vectorizer_ngram.pkl', 'rb') as f:
    loaded_vectorizer_ngram = pickle.load(f)
with open('tfidf_matrix_ngram.pkl', 'rb') as f:
    loaded_tfidf_matrix_ngram = pickle.load(f)

In [10]:
dev_claims_texts = [preprocess_text(claim['claim_text']) for claim in dev_claims_data]
dev_claim_tfidf = vectorizer.transform(dev_claims_texts)

#dev_claim_tfidf_ngram = vectorizer_ngram.transform(dev_claims_texts)

In [11]:
evidence_texts = [preprocess_text(evidence['evidence_text']) for evidence in evidence_data[:5]]
print(evidence_texts)

['john bennet lawes english entrepreneur agricultural scientist', 'lindberg began professional career age eventually moving new york city', 'boston lady cambridge vampire weekend', 'gerald francis goyer born october professional ice hockey player played game national hockey league', 'detected abnormality oxytocinergic function schizoaffective mania postpartum psychosis ect modified oxytocin release']


In [12]:
evidence_texts = [preprocess_text(evidence['evidence_text']) for evidence in evidence_data]
evidence_tfidf = vectorizer.transform(evidence_texts)

#evidence_tfidf_ngram = vectorizer_ngram.transform(evidence_texts)

In [ ]:
n = int(1208827)
divisors = [i for i in range(1, n+1) if n % i == 0]
print("整除數：", divisors)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dev_batch_size = 77 #154/2
evidence_batch_size = 1208827 #1208827/37=32671
num_dev_batches = int(np.ceil(len(dev_claims_data) / dev_batch_size))
num_evidence_batches = int(np.ceil(evidence_tfidf.shape[0] / evidence_batch_size))

dev_evidences_pred_cos = []

for i in range(num_dev_batches):
    start_dev_idx = i * dev_batch_size
    end_dev_idx = min((i + 1) * dev_batch_size, len(dev_claims_data))
    dev_batch = dev_claim_tfidf[start_dev_idx:end_dev_idx]

    for j in range(num_evidence_batches):
        start_evi_idx = j * evidence_batch_size
        end_evi_idx = min((j + 1) * evidence_batch_size, evidence_tfidf.shape[0])
        evidence_batch = evidence_tfidf[start_evi_idx:end_evi_idx]
        
        sim_matrix = cosine_similarity(dev_batch, evidence_batch)

        for k in range(sim_matrix.shape[0]):
            dev_idx = start_dev_idx + k
            top_indices = np.argsort(sim_matrix[k])[-3:][::-1]
            top_evidences_ids = [evidence_data[start_evi_idx + idx]['evidence_ID'] for idx in top_indices]
            top_similarities = [sim_matrix[k][idx] for idx in top_indices]
            dev_evidences_pred_cos.append({
                'dev_claim_ID': dev_claims_data[dev_idx]['claim_ID'],
                'evidence_IDs': top_evidences_ids,
                'evidence_sims': top_similarities
            })

In [ ]:
output = pd.DataFrame(dev_evidences_pred_cos)
output = output.drop_duplicates(subset=['dev_claim_ID'])
output.to_csv('dev_evidences_pred_cos.csv', index=False)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

dev_batch_size = 77 #154/2
evidence_batch_size = 1208827 #1208827/37=32671
num_dev_batches = int(np.ceil(len(dev_claims_data) / dev_batch_size))
num_evidence_batches = int(np.ceil(evidence_tfidf_ngram.shape[0] / evidence_batch_size))

dev_evidences_pred_cos_ngram = []

for i in range(num_dev_batches):
    start_dev_idx = i * dev_batch_size
    end_dev_idx = min((i + 1) * dev_batch_size, len(dev_claims_data))
    dev_batch = dev_claim_tfidf_ngram[start_dev_idx:end_dev_idx]

    for j in range(num_evidence_batches):
        start_evi_idx = j * evidence_batch_size
        end_evi_idx = min((j + 1) * evidence_batch_size, evidence_tfidf_ngram.shape[0])
        evidence_batch = evidence_tfidf_ngram[start_evi_idx:end_evi_idx]
        
        sim_matrix = cosine_similarity(dev_batch, evidence_batch)

        for k in range(sim_matrix.shape[0]):
            dev_idx = start_dev_idx + k
            top_indices = np.argsort(sim_matrix[k])[-5:][::-1]
            top_evidences_ids = [evidence_data[start_evi_idx + idx]['evidence_ID'] for idx in top_indices]
            top_similarities = [sim_matrix[k][idx] for idx in top_indices]
            dev_evidences_pred_cos_ngram.append({
                'dev_claim_ID': dev_claims_data[dev_idx]['claim_ID'],
                'evidence_IDs': top_evidences_ids,
                'evidence_sims': top_similarities
            })

In [ ]:
output = pd.DataFrame(dev_evidences_pred_cos_ngram)
output = output.drop_duplicates(subset=['dev_claim_ID'])
output.to_csv('dev_evidences_pred_cos_ngram1_2.csv', index=False)

In [ ]:
print("\nCliam: ")
print(dev_claims_df['claim-752'][0]['claim_text'])
print("\nReal result: ")
print(evidence_df['evidence-67732'][0])
print(evidence_df['evidence-572512'][0])
print("\nTrain - tfidf + ngram + cosine: ")
print(evidence_df['evidence-995049'][0])
print(evidence_df['evidence-554677'][0])
print(evidence_df['evidence-1140856'][0])
print(evidence_df['evidence-511300'][0])
print(evidence_df['evidence-252686'][0])

print("\nTrain - tfidf + cosine: ")
print(evidence_df['evidence-421845'][0])
print(evidence_df['evidence-554677'][0])
print(evidence_df['evidence-684667'][0])
print(evidence_df['evidence-252686'][0])
print(evidence_df['evidence-580844'][0])

print("\nTrain - jaccob: ")
print(evidence_df['evidence-572512'][0])
print(evidence_df['evidence-511300'][0])
print(evidence_df['evidence-185267'][0])
print(evidence_df['evidence-240255'][0])
print(evidence_df['evidence-252686'][0])


In [ ]:
print(evidence_df['evidence-545412'][0])

In [ ]:
print("\nCliam: ")
print(dev_claims_df['claim-752'][0]['claim_text'])
print("\nReal result: ")
print(evidence_df['evidence-67732'][0])
print(evidence_df['evidence-572512'][0])
print("\nTrain - BM25: ")
print(evidence_df['evidence-67732'][0])
print(evidence_df['evidence-572512'][0])
print(evidence_df['evidence-147175'][0])
print(evidence_df['evidence-65625'][0])
print(evidence_df['evidence-684667'][0])

In [ ]:
print(dev_claims_texts[0])
print(evidence_texts[67732])
print(evidence_tfidf[67732])

In [ ]:
test_texts = [dev_claims_texts[0], evidence_texts[572512]]
test_tfidf_matrix = vectorizer.transform(test_texts)
test_similarity_score = cosine_similarity(test_tfidf_matrix[0:1], test_tfidf_matrix[1:2])
print(test_similarity_score)

test_texts2 = [dev_claims_texts[0], evidence_texts[995049]]
test_tfidf_matrix2 = vectorizer.transform(test_texts2)
test_similarity_score2 = cosine_similarity(test_tfidf_matrix2[0:1], test_tfidf_matrix2[1:2])
print(test_similarity_score2)

In [14]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)
'''
test_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[67732].split()))
print(test_jaccard_similarity)

test_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[572512].split()))
print(test_jaccard_similarity)

test_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[421845].split()))
print(test_jaccard_similarity)

test_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[995049].split()))
print(test_jaccard_similarity)
'''

'\ntest_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[67732].split()))\nprint(test_jaccard_similarity)\n\ntest_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[572512].split()))\nprint(test_jaccard_similarity)\n\ntest_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[421845].split()))\nprint(test_jaccard_similarity)\n\ntest_jaccard_similarity = jaccard_similarity(set(dev_claims_texts[0].split()), set(evidence_texts[995049].split()))\nprint(test_jaccard_similarity)\n'

In [16]:
def get_word_set_from_tfidf(tfidf_matrix, vectorizer):
    feature_names = vectorizer.get_feature_names_out()
    word_sets = []
    for i in range(tfidf_matrix.shape[0]):
        words = [feature_names[j] for j in tfidf_matrix[i].indices]
        word_sets.append(set(words))
    return word_sets

# 轉換 dev_claims 和 evidence_texts 成詞集
dev_claim_word_sets = get_word_set_from_tfidf(dev_claim_tfidf, vectorizer)
evidence_word_sets = get_word_set_from_tfidf(evidence_tfidf, vectorizer)

#dev_claim_word_sets_ngram = get_word_set_from_tfidf(dev_claim_tfidf_ngram, vectorizer_ngram)
#evidence_word_sets_ngram = get_word_set_from_tfidf(evidence_tfidf_ngram, vectorizer_ngram)

In [ ]:
with open('dev_claim_word_sets.pkl', 'wb') as f:
    pickle.dump(dev_claim_word_sets, f)
with open('evidence_word_sets.pkl', 'wb') as f:
    pickle.dump(evidence_word_sets, f)
with open('dev_claim_word_sets_ngram.pkl', 'wb') as f:
    pickle.dump(dev_claim_word_sets_ngram, f)
with open('evidence_word_sets_ngram.pkl', 'wb') as f:
    pickle.dump(evidence_word_sets_ngram, f)

In [ ]:
with open('dev_claim_word_sets.pkl', 'rb') as f:
    loaded_dev_claim_word_sets = pickle.load(f)
with open('evidence_word_sets.pkl', 'rb') as f:
    loaded_evidence_word_sets = pickle.load(f)
with open('dev_claim_word_sets_ngram.pkl', 'rb') as f:
    loaded_dev_claim_word_sets_ngram = pickle.load(f)
with open('evidence_word_sets_ngram.pkl', 'rb') as f:
    loaded_evidence_word_sets_ngram = pickle.load(f)

In [17]:
import numpy as np

# 設定批處理大小
evidence_batch_size = 1000  # 根據你的內存設定

# 初始化存儲結構
dev_evidences_top5 = [[] for _ in range(len(dev_claims_data))]

# 分批處理證據集
for start_idx in range(0, len(evidence_word_sets), evidence_batch_size):
    end_idx = min(start_idx + evidence_batch_size, len(evidence_word_sets))
    evidence_batch = evidence_word_sets[start_idx:end_idx]

    # 計算每個 dev_claim 與當前批次中的每個 evidence 的相似度
    for i, claim_set in enumerate(dev_claim_word_sets):
        for j, evidence_set in enumerate(evidence_batch, start=start_idx):
            similarity = jaccard_similarity(claim_set, evidence_set)

            # 維護一個長度為5的最高相似度列表
            if len(dev_evidences_top5[i]) < 3:
                dev_evidences_top5[i].append((similarity, evidence_data[j]['evidence_ID']))
                dev_evidences_top5[i].sort(reverse=True, key=lambda x: x[0])
            else:
                if similarity > dev_evidences_top5[i][-1][0]:
                    dev_evidences_top5[i][-1] = (similarity, evidence_data[j]['evidence_ID'])
                    dev_evidences_top5[i].sort(reverse=True, key=lambda x: x[0])

# 格式化最終結果
dev_evidences_pred_jacob = [{
    'dev_claim_ID': dev_claims_data[i]['claim_ID'],
    'evidence_IDs': [e[1] for e in top5],
    'evidence_sims': [e[0] for e in top5]
} for i, top5 in enumerate(dev_evidences_top5)]

# 確保結果按相似度排序
'''
for entry in dev_evidences_jacob_pred:
    sorted_indices = np.argsort([-sim for sim in entry['evidence_sims']])
    entry['evidence_IDs'] = [entry['evidence_IDs'][idx] for idx in sorted_indices]
    entry['evidence_sims'] = [entry['evidence_sims'][idx] for idx in sorted_indices]
'''

"\nfor entry in dev_evidences_jacob_pred:\n    sorted_indices = np.argsort([-sim for sim in entry['evidence_sims']])\n    entry['evidence_IDs'] = [entry['evidence_IDs'][idx] for idx in sorted_indices]\n    entry['evidence_sims'] = [entry['evidence_sims'][idx] for idx in sorted_indices]\n"

In [ ]:
output = pd.DataFrame(dev_evidences_pred_jacob)
output = output.drop_duplicates(subset=['dev_claim_ID'])
output.to_csv('dev_evidences_pred_jacob_15.csv', index=False)

In [ ]:
import numpy as np

# 設定批處理大小
evidence_batch_size = 1000  # 根據你的內存設定

# 初始化存儲結構
dev_evidences_top5 = [[] for _ in range(len(dev_claims_data))]

# 分批處理證據集
for start_idx in range(0, len(evidence_word_sets_ngram), evidence_batch_size):
    end_idx = min(start_idx + evidence_batch_size, len(evidence_word_sets_ngram))
    evidence_batch = evidence_word_sets_ngram[start_idx:end_idx]

    # 計算每個 dev_claim 與當前批次中的每個 evidence 的相似度
    for i, claim_set in enumerate(dev_claim_word_sets_ngram):
        for j, evidence_set in enumerate(evidence_batch, start=start_idx):
            similarity = jaccard_similarity(claim_set, evidence_set)

            # 維護一個長度為5的最高相似度列表
            if len(dev_evidences_top5[i]) < 5:
                dev_evidences_top5[i].append((similarity, evidence_data[j]['evidence_ID']))
                dev_evidences_top5[i].sort(reverse=True, key=lambda x: x[0])
            else:
                if similarity > dev_evidences_top5[i][-1][0]:
                    dev_evidences_top5[i][-1] = (similarity, evidence_data[j]['evidence_ID'])
                    dev_evidences_top5[i].sort(reverse=True, key=lambda x: x[0])

# 格式化最終結果
dev_evidences_pred_jacob_ngram = [{
    'dev_claim_ID': dev_claims_data[i]['claim_ID'],
    'evidence_IDs': [e[1] for e in top5],
    'evidence_sims': [e[0] for e in top5]
} for i, top5 in enumerate(dev_evidences_top5)]

# 確保結果按相似度排序
'''
for entry in dev_evidences_jacob_pred:
    sorted_indices = np.argsort([-sim for sim in entry['evidence_sims']])
    entry['evidence_IDs'] = [entry['evidence_IDs'][idx] for idx in sorted_indices]
    entry['evidence_sims'] = [entry['evidence_sims'][idx] for idx in sorted_indices]
'''

In [ ]:
output = pd.DataFrame(dev_evidences_pred_jacob_ngram)
output = output.drop_duplicates(subset=['dev_claim_ID'])
output.to_csv('dev_evidences_pred_jacob_ngram.csv', index=False)

In [ ]:
with open('dev_evidences_pred_jacob.pkl', 'wb') as f:
    pickle.dump(dev_evidences_pred_jacob, f)
with open('dev_evidences_pred_jacob_ngram.pkl', 'wb') as f:
    pickle.dump(dev_evidences_pred_jacob_ngram, f)

In [ ]:
with open('dev_evidences_pred_jacob.pkl', 'rb') as f:
    loaded_dev_evidences_pred_jacob = pickle.load(f)
with open('dev_evidences_pred_jacob_ngram.pkl', 'rb') as f:
    loaded_dev_evidences_pred_jacob_ngram = pickle.load(f)

In [ ]:
dev_claims_df = pd.DataFrame(dev_claims_data)


correct_evidences = dev_claims_df.groupby('claim_ID')['evidences'].apply(list).to_dict()

def calculate_accuracy(predictions, correct_evidences):
    top_5_accuracy_count = 0
    total_claims = len(predictions)
    
    for pred in predictions:
        claim_id = pred['dev_claim_ID']
        predicted_evidences = pred['evidence_IDs']
        correct_evidences_list = correct_evidences.get(claim_id, [])
        
        set_predicted_top5 = set(predicted_evidences[:5])
        set_correct = set(correct_evidences_list)
        if set_predicted_top5 & set_correct:
            top_5_accuracy_count += 1

    top_5_accuracy = top_5_accuracy_count / total_claims if total_claims > 0 else 0

    return  top_5_accuracy

# 调用函数计算准确率
acc_cos =  calculate_accuracy(dev_evidences_pred_cos_ngram, correct_evidences)
acc_cos_ngram =  calculate_accuracy(dev_evidences_pred_cos_ngram, correct_evidences)
acc_jacob = calculate_accuracy(dev_evidences_pred_jacob, correct_evidences)
acc_jacob_ngram  = calculate_accuracy(dev_evidences_pred_jacob_ngram , correct_evidences)
#print("Cosine Accuracy:", acc_cos)  #0.2219959266802444
#print("Cosine Accuracy(ngram):", acc_cos_ngram) #0.2219959266802444
#print("Jacob Accuracy:", acc_jacob) #0.319755600814664
print("Jacob Accuracy_15:", acc_jacob)
#print("Jacob Accuracy(ngram):", acc_jacob_ngram) #0.25661914460285135

In [ ]:
print(dev_claim_word_sets)
print(pd.DataFrame(dev_evidences_pred_jacob))

In [18]:
import numpy as np
# selection: {claim_id: [evidence_id]}
# target_file: "data/dev-claims.json" or "data/train-claims.json" or "data/claims.json"
# return: recall, precision, F1

def eval_selection(selection, target_file):
    with open(target_file, 'r') as file:
        claims = json.load(file)
    selection_total = 0
    target_total = 0
    correct = 0
    for claim_id, evidence_ids in selection.items():
        claim = claims[str(claim_id)]
        for evidence_id in evidence_ids:
            if f"{evidence_id}" in claim["evidences"]:
                correct += 1
        selection_total += len(evidence_ids)
        target_total += len(claim["evidences"])
    recall = correct / target_total
    precision = correct / selection_total
    F1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else np.nan
    return recall, precision, F1



In [19]:
selection_c = {}

for item in dev_evidences_pred_cos:
    claim_id = item['dev_claim_ID']
    evidence_ids = item['evidence_IDs']

    selection_c[claim_id] = evidence_ids

print(selection_c)

{'claim-752': ['evidence-421845', 'evidence-554677', 'evidence-684667'], 'claim-375': ['evidence-1140012', 'evidence-1011788', 'evidence-359256'], 'claim-1266': ['evidence-694262', 'evidence-38305', 'evidence-943875'], 'claim-871': ['evidence-336512', 'evidence-472751', 'evidence-617077'], 'claim-2164': ['evidence-962481', 'evidence-1200633', 'evidence-776422'], 'claim-1607': ['evidence-67425', 'evidence-668884', 'evidence-57537'], 'claim-761': ['evidence-868602', 'evidence-351236', 'evidence-196050'], 'claim-1718': ['evidence-1104838', 'evidence-1186951', 'evidence-545544'], 'claim-1273': ['evidence-107843', 'evidence-1154653', 'evidence-789848'], 'claim-1786': ['evidence-124720', 'evidence-231901', 'evidence-875286'], 'claim-2796': ['evidence-403673', 'evidence-720736', 'evidence-178206'], 'claim-2580': ['evidence-208053', 'evidence-425667', 'evidence-15399'], 'claim-1219': ['evidence-1088478', 'evidence-797505', 'evidence-1115033'], 'claim-75': ['evidence-421870', 'evidence-113679',

In [20]:
selection_j = {}

for item in dev_evidences_pred_jacob:
    claim_id = item['dev_claim_ID']
    evidence_ids = item['evidence_IDs']

    selection_j[claim_id] = evidence_ids

print(selection_j)

{'claim-752': ['evidence-572512', 'evidence-511300', 'evidence-185267'], 'claim-375': ['evidence-864147', 'evidence-631053', 'evidence-778407'], 'claim-1266': ['evidence-694262', 'evidence-402339', 'evidence-1207903'], 'claim-871': ['evidence-366901', 'evidence-617077', 'evidence-1017572'], 'claim-2164': ['evidence-52981', 'evidence-1134268', 'evidence-399454'], 'claim-1607': ['evidence-668884', 'evidence-397504', 'evidence-411975'], 'claim-761': ['evidence-664808', 'evidence-620340', 'evidence-919'], 'claim-1718': ['evidence-545018', 'evidence-1104838', 'evidence-45045'], 'claim-1273': ['evidence-107843', 'evidence-789848', 'evidence-1154653'], 'claim-1786': ['evidence-714132', 'evidence-875286', 'evidence-392862'], 'claim-2796': ['evidence-632574', 'evidence-1165029', 'evidence-620691'], 'claim-2580': ['evidence-91099', 'evidence-15399', 'evidence-425667'], 'claim-1219': ['evidence-118577', 'evidence-1088478', 'evidence-797505'], 'claim-75': ['evidence-421870', 'evidence-1186875', 'e

In [22]:
recall_c, precision_c, F1_c = eval_selection(selection_c, "dev-claims.json")
print("Cosine similarity (Top 3 evidence)")
print(f"Recall: {recall_c}")
print(f"Precision: {precision_c}")
print(f"F1 Score: {F1_c}")

print("\n")
recall, precision, F1 = eval_selection(selection_j, "dev-claims.json")
print("Jaccard similarity (Top 3 evidence)")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {F1}")

Cosine similarity (Top 3 evidence)
Recall: 0.0855397148676171
Precision: 0.09090909090909091
F1 Score: 0.08814270724029381


Jaccard similarity (Top 3 evidence)
Recall: 0.08757637474541752
Precision: 0.09307359307359307
F1 Score: 0.09024134312696747


## BM25

In [5]:
evidence_data = []

for col in evidence_df.columns:
    evidence_text = evidence_df[col][0]
    evidence_id = col
    evidence_data.append({
        "evidence_ID": evidence_id,
        "evidence_text": evidence_text
    })

dev_claims_data = []
for claim_id, claim_info in dev_claims_df.items():
        update_info = {
            "claim_ID": claim_id,
            "claim_text": claim_info[0]['claim_text'],
            "claim_label": claim_info[0]['claim_label'],
            "evidences": claim_info[0]['evidences']
        }
        dev_claims_data.append(update_info)
dev_label_counts = pd.DataFrame(dev_claims_data)['claim_label'].value_counts()


claims_texts = [preprocess_text(claim['claim_text']) for claim in train_claims_data]
evidence_texts = [preprocess_text(evidence['evidence_text']) for evidence in evidence_data]
#all_texts = claims_texts + evidence_texts

NameError: name 'train_claims_data' is not defined

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.metrics import f1_score

# 初始化BM25模型
bm25 = BM25Okapi(evidence_texts, k1=0.6, b=0.6)

def get_top_n_evidence(claim_text, top_n=3):
    scores = bm25.get_scores(claim_text)
    top_indexes = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n]
    return top_indexes


NameError: name 'evidence_texts' is not defined

In [31]:
true_labels = []
pred_labels = []

for claim in dev_claims_data:
    claim_text = claim['claim_text']
    top_evidences = get_top_n_evidence(claim_text, top_n=3)
    
    # true
    true_labels.append(claim['evidences'])
    
    # pred
    pred_labels.append([evidence_data[idx]['evidence_ID'] for idx in top_evidences])

# F1 score
flatten_true_labels = [item for sublist in true_labels for item in sublist]
flatten_pred_labels = [item for sublist in pred_labels for item in sublist]


f1 = f1_score(flatten_true_labels, flatten_pred_labels, average='micro')
print(f'F1 Score: {f1:.4f}')